In [1]:
# %matplotlib inline
import ROOT as R
R.gROOT.SetBatch(True)
R.gStyle.SetOptFit( 111 )

Welcome to ROOTaaS 6.06/02


In [2]:
def windowFor( h2, tId, **kwargs ) :

#     ccc = R.TCanvas( "ccc", "ccc", 600, 600 )
    R.gStyle.SetOptFit( 111 )
    signalWidth = 85
    #get the slice
    hs = h2.ProjectionY( "tray_" + str(tId), tId, tId )
    
    size = 100
    if "size" in kwargs :
        size = kwargs["size"]
    
    
    
    
    # look at the peak
    hs.GetXaxis().SetRange( hs.GetMaximumBin() - size, hs.GetMaximumBin() + size )
    ff = R.TF1( "ff", "landau", hs.GetMaximumBin() - 50, hs.GetMaximumBin() + 50 )
    ff.SetLineColor( R.kRed )
    ff.SetLineWidth( 3 )
    hs.Fit( ff, "RQ" )
    
    
    cutLow = ff.GetParameter(1) - 25
    cutHigh = ff.GetParameter(1) + signalWidth
    #print "par1=", ff.GetParameter(1)
    hs.Draw( );
    
#     print "cutLow", cutLow
    hMax = hs.GetMaximum() * 2
#     print "max =", hMax
    l1 = R.TLine( 1050, 1, 1050, 200 )
    l1.SetLineColor( R.kBlue )
    l1.SetLineWidth( 3 )
#     l2 = R.TLine( cutHigh, 1, cutHigh, hMax )
#     l2.SetLineColor( R.kBlue )
#     l2.SetLineWidth( 3 )
    l1.Draw( )
#     l2.Draw( )


#     if "title" in kwargs :
#         hs.SetTitle( kwargs[ "title"] )
#     hs.GetXaxis().SetTitle( "" )
#     hs.GetYaxis().SetTitle( "[#]" )
    
#     hs.SetAxisRange( cutLow - size, cutHigh + size, "X" )
#     R.gPad.SetLogy(0)
#     ccc.Draw()
#     ccc.Print( hs.GetTitle() + ".pdf" )
    if hMax < 1 :
        print "tray ", tId, " is dead"
        
    return int(cutLow), int(cutHigh)

In [3]:
def writeTriggerWindows( lowEdges, highEdges ) :
    fname ="triggerWindows_" + rn + ".dat"
    print "Exporting trigger windows to ", fname
    target = open( fname, "w" )
    target.write( "double cutlow[122] ={\n" )
    for i in range( 1, 123 ):
        if i != 122 :
            target.write("%i,"% lowEdges[ i-1 ])
        else :
            target.write("%i };"% lowEdges[ i-1 ])
        if i % 10 == 0:
            target.write("\n")
            
    target.write("\n\n")
    target.write("double cuthi[122] ={\n")
    for i in range( 1, 123 ):
        if i != 122 :
            target.write("%i,"% highEdges[ i-1 ])
        else :
            target.write("%i };"% highEdges[ i-1 ])
        if i % 10 == 0:
            target.write("\n")
    target.close()

In [5]:
def makeReport( **kwarg ) :
    
    leTrg = data.Get("leTrg")
    trTrg = data.Get("TrTrg")

    can = R.TCanvas( "c1", "c1", 2000, 1200 )
    can.Clear()
    name = "triggerWindows_" + rn + ".pdf"
    if "name" in kwarg :
        name = kwarg[ "name" ]
    print "Generating PDF report at ", name
    can.Print( name + "[" )
    ndivX = 1
    ndivY = 1
    ndiv = ndivX * ndivY
    can.Divide( ndivX, ndivY )
    lowEdges = []
    highEdges = []

    for tId in range( 1, 123 ):
#         can.cd( (tId-1) % ndiv + 1  )
        low, high = windowFor( leTrg, tId, size=100 )
        lowEdges.append( low )
        highEdges.append( high )
        if  ((tId-1) % ndiv == ndiv-1) or (tId == 122):
            can.Draw()
            can.Print( name )
            can.Clear()
            can.Divide( ndivX, ndivY )
    
    can.Print( name + "]" )
    
    writeTriggerWindows(lowEdges, highEdges)

In [6]:
def reportTriggerWindowsInline( lowEdges, highEdges ) :
    print "double cutlow[122] ={"
    for i in range( 1, 123 ):
        if i != 122 :
            print("%i,"% lowEdges[ i-1 ]),
        else :
            print("%i };"% lowEdges[ i-1 ]),
        if i % 10 == 0:
            print ""

    print "\n"
    print "double cuthi[122] ={"
    for i in range( 1, 123 ):
        if i != 122 :
            print("%i,"% highEdges[ i-1 ]),
        else :
            print("%i };"% highEdges[ i-1 ]),
        if i % 10 == 0:
            print ""

In [14]:
def makeReportInline( page ) :
    leTrg = data.Get("leTrg")
    trTrg = data.Get("TrTrg")

    can = R.TCanvas( "c1", "c2", 2000, 1200)
    can.Clear()
    
    can.Divide( 4, 4 )
    lowEdges = []
    highEdges = []

    for tId in range( 1 + 16 * page, 1 + 16 * (page + 1) ):
        can.cd( tId - 16 * page  )
        low, high = windowFor( leTrg, tId, size=500 )
        lowEdges.append( low )
        highEdges.append( high )
    
    #if 0 == page :
    #    reportTriggerWindowsInline(lowEdges, highEdges)
    return can